# modeling12-1-get-data-for-incremetal-data-from-binary-inference

- Incremental 
- Method

In [1]:
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os

## Get dataset for incremental Test
- from the output from modeling11-nli-binaryclassifier-modeling-from-scratch-prediction

## Incremental Testing Setting

### 3 Method (Check out README.md)

### Method 2 (2nd method of in README.md)
    - The sooner, The better approach
      - Keep the Positive Context in order
    - Since we testing it, let's keep sample size of 5
    - When there is no Exact Mathcing during the incremental inference e.g.) em_pattern = '00000'
        - Keep the whole context so that those cases will have False on Exact Match values
        
    - Patterns: 
        - first 1 : positive
        - first 0 : irrelevant 
            - we know that first is the positive context that includes answer
            - with that perspective, it is relevant in theory
            - ; however, dataset is created via BLEU score on two different corpus.
            - If FiD does not correctly infer the output, 
            - context what we concieved of artificial positive context is actually irrelevant to the query.
            - Also when realistic scenario, we don't know whether the first context contains the answer
        - 01 pattern : Positive 
        - 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
        - 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
        - 10 pattern : Damaging
        
        - 11 pattern : relevant vs positive
            Strict Positive
                Under strict rule : consider it as relevant ?(or irrelevant?)
                                in terms of strictly limiting the number of positive passages
            Naive Positive
                Under naive rule : consider it as positive
                                in that this would increase the number of positive passages
                
        - A00 pattern : irrelevant vs damaging
            if '1' does not occured in A, currnet passage is irrelevant
            if '1' occurred in A, current passage is damaging either irrelevant 
                Strict Damaging
                    Under strict rule : consider it as irrelevnat 
                                    in terms of strictly limiting the number of damaging passage
                Naive Damaging
                    Under naive rule : consider it as damaging 
                                    in that this would increase the number of damaging passages
        
    - Options1 : Removes only damaging
    - Options2 : Removes damaging + irrelevant
    - Options3 : Removes damaging + relevant
    - Options4 : Removes damaging + irrelevant + relevnat
        

# Input
    - result from 5-1
    - /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json

# Trials
    - Need to check FiD input when there are less ctxs than n_context

In [2]:
sample_size = 100
input_file = f'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/prediction.json'

In [6]:
input_data = utils.open_json(input_file)

In [38]:
pprint((input_data[5]))

{'answers': ['Wilhelm Conrad Röntgen'],
 'binary_inference': '1',
 'ctx': {'id': '718073',
         'text': 'Wilhelm Röntgen Wilhelm Conrad Röntgen (; ; 27 March 1845 – '
                 '10 February 1923) was a German mechanical engineer and '
                 'physicist, who, on 8 November 1895, produced and detected '
                 'electromagnetic radiation in a wavelength range known as '
                 'X-rays or Röntgen rays, an achievement that earned him the '
                 'first Nobel Prize in Physics in 1901. In honour of his '
                 'accomplishments, in 2004 the International Union of Pure and '
                 'Applied Chemistry (IUPAC) named element 111, roentgenium, a '
                 'radioactive element with multiple unstable isotopes, after '
                 'him. Born to a German father and a Dutch mother, Röntgen '
                 'attended high school in Utrecht, Netherlands. In',
         'title': 'Wilhelm Röntgen'},
 'em': '1',
 'gold':

In [22]:
len(input_data)

361000

In [ ]:
def get_prediction_em_pattern(input_data):

In [23]:
0 in predict_id

False

In [39]:
predict_id = {}
for ins in input_data:
    if ins['id'] not in predict_id:
        predict_id[ins['id']] = str(ins['binary_inference'])
    else:
        predict_id[ins['id']] += str(ins['binary_inference'])

In [40]:
len(predict_id)

3610

### format
```python
{
    'id' : str()
    'answers' : list()
    'ctxs' : list(dict)
    'questions' : str()
}
```

In [14]:
test_inferred = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR/ctx100id.json'

In [15]:
test_data_inferred = utils.open_json(test_inferred)

In [59]:
# predict_id[2]

In [48]:
test_data_inferred[0].keys()

dict_keys(['question', 'answers', 'ctxs', 'em_pattern', 'inference', 'conversion', 'cumulative_em', 'id'])

In [49]:
for ins in test_data_inferred:
    prediction_em_pattern = predict_id[ins['id']]
    ins['prediction_em_pattern'] = prediction_em_pattern
    

In [53]:
test_data_inferred[1].keys()

dict_keys(['question', 'answers', 'ctxs', 'em_pattern', 'inference', 'conversion', 'cumulative_em', 'id', 'prediction_em_pattern'])

In [57]:
output_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction'
output_file_name = 'ctx100id_prediction_em.json'
output_file_path = os.path.join(output_path, output_file_name)
print(output_file_path)

/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/ctx100id_prediction_em.json


In [58]:
utils.save_json(test_data_inferred, output_file_path)

# Implementation

In [74]:
def build_data_on_prediction(input_file, option, option_p, option_d, sample_size):
    '''
    input_file : incremental inference result from FiD from KILT-5-1
        path : /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json
        
    output : FiD input json format
    
    option(required) : removing strategies
        op1 : removes damages only
        op2 : removes damaging + irrelevant
        op3 : removes damaging + relevant
        op4 : removes damaging + irrelevant + relevant
        
    option_p(required) : positive passage selection options
        strict : strict positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is relevant
        naive : naive positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is positive
                
    option_d(required) : damaging passage selection options
        strict : strict negative
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is irrelevant
        naive : naive damaging
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is damaging
    
    '''
    
    output_format = []
    null_em = '0' * sample_size
    
    # 'strict', 'naive'  
    # option_p = 'naive'
    # option_d = 'naive'
    # option = 'op4'

    for id_, instance in enumerate(input_file,1):
        template_dict = {}
        if 'id' in instance.keys():
            template_dict['id'] = instance['id']
        else:
            template_dict['id'] = str(id_)
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
#         template_dict['em_pattern'] = instance['em_pattern']
        template_dict['em_pattern'] = instance['em_pattern']
        template_dict['prediction_em_pattern'] = instance['prediction_em_pattern']

        em_pattern = instance['prediction_em_pattern']

        # when there is at least one EM in the accumulated inference
        if em_pattern != null_em:   
            new_ctx = []

            # relevant vs positive
            positve_ctx_lst = []
            relevant_ctx_lst = []

            # irrelevant vs damaging
            damaging_ctx_lst = []
            irrelevant_ctx_lst = []


            for idx_, ctx in enumerate(instance['ctxs']):

                # checking current em
                cur_em = em_pattern[idx_]
                pre_em_pattern = em_pattern[:idx_]


                # first 1 : positive
                if not pre_em_pattern and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # first 0 : irrelevant
                elif not pre_em_pattern and cur_em == '0':
                    irrelevant_ctx_lst.append(ctx)
                    
                # 01 pattern : positive 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # 10 pattern : damaging
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
                    damaging_ctx_lst.append(ctx)

                # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
                    if option_p == 'strict':
                        relevant_ctx_lst.append(ctx)

                    elif option_p == 'naive':
                        positve_ctx_lst.append(ctx)

                    else:
                        print('option_p should be either \'strict\' or \'naive\'')
                        return 

                # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
                    # if '1' does not occured in A, currnet passage is irrelevant
                    if not '1' in pre_em_pattern:
                        irrelevant_ctx_lst.append(ctx)

                    # if '1' occurred in A, 
                    else:
                        # strict : consider it as irrelevnat 
                        if option_d == 'strict':
                            irrelevant_ctx_lst.append(ctx)

                        # naive : consider it as damaging 
                        elif option_d == 'naive':
                            damaging_ctx_lst.append(ctx)

                        else:
                            print('option_p should be either \'strict\' or \'naive\'')
                            return 

            # op1 removes damages only
            if option == 'op1':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)


            # op2 removes damaging + irrelevant
            elif option == 'op2':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)

            # op3 : Removes damaging + relevant
            elif option == 'op3':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)

            # op4 : Removes damaging + irrelevant + relevant
            elif option == 'op4':
                new_ctx.extend(positve_ctx_lst)

            else:
                print('option should be op1, op2, op3, op4')
                return 

            template_dict['ctxs'] = new_ctx
            output_format.append(template_dict)

        # when there is no EM in the accumulated inference
        else:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
    
    print('==============instance finished======================')
    return output_format
        


In [6]:
# option_dict = {
#     'op1' : 'remove_damage',
#     'op2' : 'remove_damage_irrelevant',
#     'op3' : 'remove_damage_relevant',
#     'op4' : 'remove_damage_irrelevant_relevant',
# }
# option = 'op4'

In [62]:
# op1, op2 are duplicates
method_option_dict = {
    'op3' : 'remove_damage_relevant',
    'op4' : 'remove_damage_irrelevant_relevant',
}

In [63]:
option_p_dict = {
    'strict' : 'strict_positive',
    'naive' : 'naive_positive',
}
# option_p = 'strict'

In [64]:
option_d_dict = {
    'strict' : 'strict_damaging',
    'naive' : 'naive_damaging',
}
# option_d = 'strict'

In [10]:
# input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')
# output_path = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION'

# TEST set Inference

In [60]:
input_file = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/ctx100id_prediction_em.json'

In [61]:
input_ = utils.open_json(input_file)

In [66]:
len(input_)

3610

In [68]:
output_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods'

In [69]:
output_path

'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods'

In [72]:
os.makedirs(output_path, exist_ok = True)

In [76]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
            output_format = build_data_on_prediction(input_, option, option_p, option_d, sample_size)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

==============instance finished======================
strict_positive_strict_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods
==============instance finished======================
strict_positive_naive_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods
==============instance finished======================
naive_positive_strict_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods
==============instance finished======================
naive_positive_naive_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/

In [90]:
test_path= '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1/step_320/intact_prediction/selection_methods'
# method 2
test_file = 'strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json'
test_file_path = os.path.join(test_path, test_file)

In [91]:
test_data = utils.open_json(test_file_path)

In [92]:
len(test_data)

3610

In [93]:
test_data[0].keys()

dict_keys(['id', 'answers', 'question', 'em_pattern', 'prediction_em_pattern', 'ctxs'])

In [94]:
for instance in test_data:
    print(instance['em_pattern'])
    print(instance['prediction_em_pattern'])
    print(len(instance['ctxs']))
    print('----')

0000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1000110010000000001100000000000000000000000000000000000000000000000000000000000000000000010000000000
5
----
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1000011000111011110000100110001110000001010000000010110000010000000111110101000000011000001000000010
18
----
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000
1
----
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0100110000000000000000000000000001000000100000000000000000000000000000000000000000000000000000000000
4
----
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
10000000010000000000000100000001000000000000000000000000000000

## NQ dev set

In [11]:
sample_size = 100
input_file = f'/data/philhoon-relevance/FiD/results/NQ_DPR/DEV/incremental_result_{sample_size}/ctx{sample_size}.json'
output_path = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_{sample_size}'

In [12]:
input_ = utils.open_json(input_file)

In [13]:
output_path

'/data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_100'

In [14]:
len(input_)

8757

In [15]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
            output_format = build_data(input_, option, option_p, option_d, sample_size)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

==============instance finished======================
strict_positive_strict_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_100
==============instance finished======================
strict_positive_naive_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_100
==============instance finished======================
naive_positive_strict_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_100
==============instance finished======================
naive_positive_naive_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_100
==============instance finished======================
strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_100

# Method explanation
- Redundancies in selection strategies -> remove them -> 6

#### Method1. Include passages that corresponds to 1s
- all __option2__ belong to this case
- option4_naive_naive, option4_naive_strict

'00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000  
- red 1s

#### Method2. Include passages that corresponds to first 1s
- option4_strict_naive, option4_strict_strcit 

'00011011000'  -> 
000<span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s
- removing consecutive passages when the previous output is correct as well

#### Method3. Include passages that corresponds to 1s + Included First appeared consecutive 0s
- option1_naive_naive, option1_strict_naive
- option3_naive_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  
- red 1s and blue 0s
- usually top-retrieved results contain the answer 

#### Method4. Include passages that corresponds to first 1s + Included First appeared consecutive 0s
- option3_strict_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- usually top-retrieved results contain the answer 

#### Method5. Include passages that corresponds to 1s + Remove only damaging
- option1_naive_strict, option1_strict_strict
- option3_naive_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue"><span style="color:blue">0</span><span style="color:blue"> 
- red 1s and blue 0s
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

#### Method6. Include passages that corresponds to first 1s  + Remove only damaging
- option3_strict_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span>
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

## Conclusion
- option1, option2 are not needed
- Method1 : option4_naive_naive
- Method2 : option4_strict_naive 
- Method3 : option3_naive_naive
- Method4 : option3_strict_naive
- Method5 : option3_naive_strict
- Method6 : option3_strict_strict